# Mortality Forecast

## Importing Libraries

In [1]:
import datetime
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers

## Setting Environment

In [2]:
TRAINING_SIZE = 49  # number of observations used for training
WINDOW = 3  # the length of the sequence used for prediction
CONDITIONAL_RNN = True  # whether we use conditional RNN (by gender and country)
tf.random.set_seed(1918)

## Data Preprocessing

In [4]:
def read_data(filenames):
    dataframes = []
    for filename in filenames:
        dataframes.append(
            pd.read_csv(filename, header=1, sep=r'\s+'))
    return dataframes


def long_to_wide(dataset):
    dataset = dataset.melt(
        id_vars=["Year", "Age"], value_vars=["Male", "Female"])
    dataset = dataset.pivot(
        columns=["Age"], index=["Year", "variable"], values=["value"])
    dataset = dataset.reset_index()
    dataset["variable"] = dataset["variable"] == "Female"
    dataset = dataset.astype('float')
    dataset["Year"] = dataset["Year"].transform(
        lambda x: datetime.date(int(x), 1, 1))
    dataset.columns = dataset.columns.get_level_values(1)
    dataset.columns.values[0] = "Year"
    dataset.columns.values[1] = "F"
    dataset = dataset.set_index(dataset.columns[0])
    dataset = dataset.astype('float')
    dataset = dataset.sort_values(by=['F', 'Year'])
    return dataset


def clean_data(dataframes):
    clean_dataframes = []
    for dataframe in dataframes:
        dataframe = dataframe.drop(columns="Total")
        dataframe = long_to_wide(dataframe)
        clean_dataframes.append(dataframe)
    return clean_dataframes


lt_mortality_raw, lt_deaths_raw, lt_exposure_raw, \
    lv_mortality_raw, lv_deaths_raw, lv_exposure_raw, \
    ee_mortality_raw, ee_deaths_raw, ee_exposure_raw = read_data(
        (
            "./data/LT_Mx_5x1.txt", "./data/LT_Deaths_5x1.txt",
            "./data/LT_Exposures_5x1.txt", "./data/LV_Mx_5x1.txt",
            "./data/LV_Deaths_5x1.txt", "./data/LV_Exposures_5x1.txt",
            "./data/EE_Mx_5x1.txt", "./data/EE_Deaths_5x1.txt",
            "./data/EE_Exposures_5x1.txt"
        ))

lt_mortality, lv_mortality, ee_mortality = \
    lt_mortality_raw.replace(".", None), lv_mortality_raw.replace(".", None), \
    ee_mortality_raw.replace(".", None)

lt_mortality, lt_deaths, lt_exposure = clean_data(
    (lt_mortality, lt_deaths_raw, lt_exposure_raw))

lv_mortality, lv_deaths, lv_exposure = clean_data(
    (lv_mortality, lv_deaths_raw, lv_exposure_raw))

ee_mortality, ee_deaths, ee_exposure = clean_data(
    (ee_mortality, ee_deaths_raw, ee_exposure_raw))

lt_mortality["LT"], lv_mortality["LV"], ee_mortality["EE"] = 1, 1, 1

lt_deaths, lt_mortality, lt_exposure = \
    lt_deaths.drop(lt_mortality.tail(1).index), \
    lt_mortality.drop(lt_mortality.tail(1).index), \
    lt_exposure.drop(lt_mortality.tail(1).index)